This workbook runs a process whereby an starting position (the opening position with White to move for now) is taken as an input.  It then builds a Python array with ALL the possible moves for a defined level of half-moves (i.e. 0 - 6).  The array generates the leaf nodes of the tree that represents the starting positions for a random play through to conclusion.  


TO DO: 
-Once the leaf node (starting position) is selected, random moves will be played through to critical state (DRAW or WIN).  
-The idea is then to propagate the reward back up the tree to the parent nodes in the game.
-Then one could aggregate the reward for each one of the branches down to the starting position and pick a next move based on the optimal reward (largest for White, smallest for Black).
-Lastly, the game tree could be transferred to a DataFrame and persisted to the Teradata database.

CHALLENGES:
-The tree explodes exponentially, and anything past a 2 half-move depth becomes prohibitively slow.

In [17]:
import chess

First we create some helper functions to work with the Python-chess library.

In [18]:
def legal_moves_array(in_board):
        # Gets all the legal moves for the board and enumerate them in an array.
        all_moves = str(in_board.legal_moves)
        start_pos = [pos for pos, char in enumerate(all_moves) if char == '(']
        end_pos = [pos for pos, char in enumerate(all_moves) if char == ')']
        all_moves_string= all_moves[start_pos[0]+1:end_pos[0]]
        all_moves_string = all_moves_string.replace(" ", "")
        all_moves_array = all_moves_string.split(',')
        
        return all_moves_array

def truncate_fen(in_board):
        # Truncates the FEN to exclude the ply and full move.
        fen = in_board.fen()
        spaces_pos = [pos for pos, char in enumerate(fen) if char == ' ']
        truncated_fen = fen[:spaces_pos[3]]
        
        return truncated_fen

The next section builds the Monte Carlo Tree from a starting position (in this case the starting position is the opening position with White making the first move).  Tthe depth of the search can be specified using the parameter depth = n.  Just note that the depth and legal variations from the opening position are shown respectively below so stay away from depth > 3'ish.

(1:20), (2:400), (3:8902), (4:197281), (5:4865609), (6:119060324) 

The leaf nodes (boards) are written to the array batch_array.  The leaf nodes of the batch_array represents the starting point of the games to be played.

In [19]:
depth = 5
node_board = chess.Board()
truncated_fen = truncate_fen(node_board)
node_index = 0

test_fen = ''

# Initialize the game_tree data.
batch_array = [
    # Node Index, Frame Level, Parent Move Index, Truncated FEN, Move, Reward
    [node_index,0,0,truncated_fen,'',0.]
]

control_array = batch_array


for d_level in range(depth):
    #print(d_level)
    frame_level = d_level + 1

    for row in control_array:
        
        if row[1] == d_level:
            node_board.set_fen(row[3])
            legal_moves = legal_moves_array(node_board)

            for move in legal_moves:
                if move != '':  # This needs to be checked in-case we get to a terminal state.  It should be augmented with DRAW states as well.
                    node_board.push_san(move)
                    truncated_fen = truncate_fen(node_board)
                    node_index = node_index + 1
                    half_move_array = [node_index, frame_level, row[0],truncated_fen, move, 0.]
                    batch_array.append(half_move_array)
                    
                    node_board.pop()

    control_array = batch_array       

In [21]:
counter = 0
print(len(batch_array))
for row in batch_array:
    if row[0] == depth:
        counter += 1
    #print(row)

print(counter)

5072213
1
